In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [13]:
def circuit_left():
    """
    This function corresponds to the circuit on the left-hand side of the diagram in the
    description. Simply place the necessary operations, you do not have to return anything.
    """
    # nullify noise (PauliX) on wires 1 and 2
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    # TODO: nullify noise on wire 0

def circuit_right():
    """
    This function corresponds to the circuit on the right-hand side of the diagram in the
    description. Simply place the necessary operations, you do not have to return anything.
    """
    # nullify noise (PauliX) on wires 1 and 2
    # HXH = Z in case of noise (PauliX), and HH = I in case of no noise.
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    # TODO: nullify noise on wire 0

    # create bell+ state
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1,2])

    # teleport
    qml.CNOT(wires=[0,1])
    qml.Hadamard(wires=0)
    m_0 = qml.measure(0)
    # TODO: pass information to wire 1 first
    qml.cond(m_0 == 1, qml.PauliZ)(wires=2)

    m_1 = qml.measure(1)
    qml.cond(m_1 == 1, qml.PauliX)(wires=2)


In [14]:
def U():
    """This operator generates a PauliX gate on a random qubit"""
    qml.PauliX(wires=np.random.randint(3))


dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def circuit(alpha, beta, gamma):
    """Total circuit joining each block.

    Args:
        alpha (float): The first parameter of a U3 gate.
        beta (float):The second parameter of a U3 gate.
        gamma (float): The third parameter of a U3 gate.

    Returns:
        (float): The expectation value of an observable.
    """
    qml.U3(alpha, beta, gamma, wires=0)
    circuit_left()
    U()
    circuit_right()

    # Here we are returning the expected value with respect to any observable,
    # the choice of observable is not important in this exercise.

    return qml.expval(0.5 * qml.PauliZ(2) - qml.PauliY(2))


In [16]:
circuit(2.0, 1.0, 3.0)

tensor(0.20807342, requires_grad=True)

In [5]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    angles = json.loads(test_case_input)
    output = circuit(*angles)
    return str(output)

def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    b = np.allclose(solution_output, expected_output, atol=2e-1)
    print(b)
    assert b, "The expected output is not quite right."

    ops = circuit.tape.operations

    for op in ops:
        assert not (0 in op.wires and 2 in op.wires), "Invalid connection between qubits."

    assert circuit.tape.observables[0].wires == qml.wires.Wires(2), "Measurement on wrong qubit."


In [6]:
test_cases = [['[2.0,1.0,3.0]', '-0.97322'], ['[-0.5,1.2,-1.2]', '0.88563'], ['[0.22,3.0,2.1]', '0.457152'], ['[2.22,3.1,-3.3]', '-0.335397'], ['[-0.2,-0.1,3.4]', '0.470199'], ['[-1.2,-1.1,0.4]', '-0.6494612']]

In [9]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        print(f"(output, expected_output): ({output},{expected_output})")
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[2.0,1.0,3.0]'...
(output, expected_output): (-0.9732208195078625,-0.97322)
True


AssertionError: Invalid connection between qubits.